In [1]:
import os
import json
import requests
import uuid
from datetime import datetime
chat_api_endpoint = os.environ.get("CHAT_API_ENDPOINT")
chat_api_key = os.environ.get("CHAT_API_KEY")

## Get an answer

In [2]:
def get_answer_api(question:str, conversationId: str = None)->dict:
    endpoint = f"{chat_api_endpoint}/chat?code={chat_api_key}"

    #generate guid
    guid = str(uuid.uuid4())
    if(conversationId == None):
        conversationId = f"ragas-{datetime.today().strftime('%Y%m%d')}-{guid}"

    data = {
        "ConversationId" : conversationId,
        "Message" : question,
        "UserId" : "RAGAS"
    }
    response = requests.post(endpoint, json=data)
    answer = response.json()["Message"]
    intent = response.json()["Question"]
    context = json.loads(response.json()["Context"])

    context_combined = []
    for c in context:
        context_combined.append(str(c["id"]) + ": " + c["content"])

    context = context_combined
    return answer, context, intent, conversationId

In [3]:
question = "Wie is de burgemeester"
answer, context, intent, conversationId = get_answer_api(question)
print(conversationId)
print(answer)
print(context)
print(intent)

ragas-20240725-eab5adad-a3b5-44ba-88da-be3002c0a9f5
De burgemeester van Kortrijk is Vincent Van Quickenborne.
['1381: Burgemeester Vincent Van Quickenborne Grote Markt 54 8500 Kortrijk België Burgemeester Vincent Van Quickenborne Veiligheid Brandweer Politie Preventie Samenleven, diversiteit en inburgering GAS Strategische planning Coördinatie stedenbeleid Externe relaties en samenwerking Eurometropool Intercommunales en samenwerking met derden Bestuurszaken Recht Communicatie', '2175: Objectieve aansprakelijkheid in geval van brand en ontploffing Objectieve aansprakelijkheid Omschrijving Dit is een verplichte verzekering voor objectieve aansprakelijkheid in geval van brand en ontploffing voor inrichtingen die gewoonlijk voor het publiek toegankelijk zijn. ( wet van 30 juli 1979 ) Het behoort de burgemeester toe te controleren of de door de wet bedoelde inrichtingen de vereiste verzekeringspolis hebben afgesloten. De burgemeester heeft de bevoegdheid om de tijdelijke sluiting op te leg

## Get my dataset

In [6]:
import pandas as pd
import datasets
data = pd.read_csv("ragas-data.csv")
print("Questions to be evaluated: ", len(data))

Questions to be evaluated:  5


In [7]:
data

,question,ground_truth
0,Wat is het adres containerpark Heule?,Het adres van het containerpark in Heule is La...
1,Wat zijn de openingsuren containerpark Rollegem?,Het containerpark in Rollegem is geopend zonde...
2,Moet ik een afspraak maken voor het containerp...,"Nee, je moet voor geen enkel containerpark een..."
3,Wat zijn de openingsuren bibliotheek kortrijk?,De bibliotheek in Kortrijk is open op weekdage...
4,wat is het adres van de bibliotheek?,Het adres van de bibliotheek is Leiestraat 30 ...


## Generate answers

In [8]:
ragas_data = list()

for i in range(len(data)):
    try:
        question = data.loc[i, "question"]
        ground_truth = data.loc[i, "ground_truth"]
        answer, context, intent, conversationId = get_answer_api(question)
        ragas_data.append({
            "question" :question,
            "ground_truth" : ground_truth,
            "answer" : answer,
            "contexts" : context,
            "intent" : intent
        })
        print(f"{i+1}/{len(data)} - {question}")
    except Exception as e:
        print(f"Error at index {i} - {e}")

ragas_data_df = pd.DataFrame(ragas_data)

1/5 - Wat is het adres containerpark Heule?
2/5 - Wat zijn de openingsuren containerpark Rollegem?
3/5 - Moet ik een afspraak maken voor het containerpark?
4/5 - Wat zijn de openingsuren bibliotheek kortrijk?
5/5 - wat is het adres van de bibliotheek?


## Evaluation

In [9]:
import ragas
import ragas.metrics as metrics
from langchain_openai.chat_models import AzureChatOpenAI
from langchain_openai.embeddings import AzureOpenAIEmbeddings

azure_model = AzureChatOpenAI(
    openai_api_version="2024-06-01",
    azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
    azure_deployment="gpt-4o",
    model="gpt-4o",
    validate_base_url=False,
    api_key=os.environ.get("AZURE_OPENAI_KEY")
)

# init the embeddings for answer_relevancy, answer_correctness and answer_similarity
azure_embeddings = AzureOpenAIEmbeddings(
    openai_api_version="2024-06-01",
    azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
    azure_deployment="text-embedding-ada-002",
    model="text-embedding-ada-002",
    api_key=os.environ.get("AZURE_OPENAI_KEY")
)

In [10]:
ragas_dataset = {
    "question" : ragas_data_df["question"].to_list(),
    "ground_truth" : ragas_data_df["ground_truth"].to_list(),
    "answer" : ragas_data_df["answer"].to_list(),
    "contexts" : ragas_data_df["contexts"].to_list(),
}
ds = datasets.Dataset.from_dict(ragas_dataset)

result = ragas.evaluate(
    dataset= ds,
    metrics=[metrics.faithfulness, 
             metrics.answer_correctness, 
             metrics.context_recall, 
             metrics.context_precision],
    llm=azure_model,
    embeddings=azure_embeddings,
    raise_exceptions=False
)
merged_df = pd.merge(ragas_data_df, pd.DataFrame(result.scores), left_index=True, right_index=True)

Evaluating:  10%|█         | 2/20 [01:05<11:29, 38.32s/it]Runner in Executor raised an exception
Runner in Executor raised an exception
Runner in Executor raised an exception
Runner in Executor raised an exception
Runner in Executor raised an exception
Runner in Executor raised an exception
Runner in Executor raised an exception
Runner in Executor raised an exception
Runner in Executor raised an exception
Runner in Executor raised an exception
Runner in Executor raised an exception
Runner in Executor raised an exception
Runner in Executor raised an exception
Runner in Executor raised an exception
Evaluating:  90%|█████████ | 18/20 [02:08<00:12,  6.47s/it]Runner in Executor raised an exception
Runner in Executor raised an exception
Evaluating: 100%|██████████| 20/20 [02:40<00:00,  8.00s/it]
c:\MyRepos\LLMSafety\.venv\Lib\site-packages\ragas\evaluation.py:296: RuntimeWarning: Mean of empty slice
  value = np.nanmean(self.scores[cn])
